In [117]:
import os
import pandas as pd
import numpy as np

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Avalon import pyAvalonTools

# 경로 설정
path = os.getcwd()
data_path = path + '/data/'
if not os.path.exists(data_path): # 디렉토리가 존재하지 않을 시 생성
    os.makedirs(data_path)
data = pd.read_csv(data_path + 'dataset.csv') # dataset.csv 부분에 new drug data 입력

smiles_list = data['SMILES'].values # data에서 smiles 부분만 추출

# SMILES 문자열을 Molecule 객체로 변환하고 Avalon fingerprint 생성
avalon_fps = []
for smiles in smiles_list: # 각각의 smile에 해당하는 avalon_fp 를 얻기 위한 for문
    mol = Chem.MolFromSmiles(smiles) # smiles를 molecule로 변환
    if mol is not None:
        avalon_fp = rdkit.Avalon.pyAvalonTools.GetAvalonFP(mol, nBits=1024) # molecule 을 avalon_fp로 변환
        binary_avalon_fp = avalon_fp.ToList() # 0,1 로 구성된 1024bit로 표현
        avalon_fps.append(binary_avalon_fp)
    else:
        print(f"Failed to generate Avalon fingerprint for SMILES: {smiles}") # SMILES 정보가 database에 존재하지 않을 시 error
avalon_fps = np.array(avalon_fps)

# avalon_fp의 각 col name을 Avalon_i 로 표현
col_list = [] 
for i in range(1024):
    col_names = f'Avalon_{i}'
    col_list.append(col_names)
    
avalon_bits = pd.DataFrame(avalon_fps)
avalon_bits.columns = col_list

# 각 데이터의 ID값과 SMILES 추출
data_id = data.iloc[:,0:2]

# Avalon 저장
Avalon_bits = pd.concat([data_id,avalon_bits], axis=1)
Avalon_bits.to_csv(data_path + 'Avalon_bits.csv',index=False)